Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Build TensorFlow Lite Support libraries with Bazel 

## Set up Android environment

In [ ]:
# Create folders
!mkdir -p '/android/sdk'

# Download and move android SDK tools to specific folders
!wget -q 'https://dl.google.com/android/repository/tools_r25.2.5-linux.zip'

!unzip 'tools_r25.2.5-linux.zip'
!mv '/content/tools' '/android/sdk'
# Copy paste the folder
!cp -r /android/sdk/tools /android/android-sdk-linux

# Download NDK, unzip and move contents
!wget 'https://dl.google.com/android/repository/android-ndk-r19c-linux-x86_64.zip'

!unzip 'android-ndk-r19c-linux-x86_64.zip'
!mv /content/android-ndk-r19c /content/ndk
!mv '/content/ndk' '/android'
# Copy paste the folder
!cp -r /android/ndk /android/android-ndk-r19c

# Remove .zip files
!rm 'tools_r25.2.5-linux.zip'
!rm 'android-ndk-r19c-linux-x86_64.zip'

# Make android ndk executable to all users
!chmod -R go=u '/android'

In [ ]:
# Set and view environment variables
%env PATH = /usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/android/sdk/tools:/android/sdk/platform-tools:/android/ndk
%env ANDROID_SDK_API_LEVEL=29
%env ANDROID_API_LEVEL=29
%env ANDROID_BUILD_TOOLS_VERSION=30.0.0
%env ANDROID_DEV_HOME=/android
%env ANDROID_NDK_API_LEVEL=21
%env ANDROID_NDK_FILENAME=android-ndk-r19c-linux-x86_64.zip
%env ANDROID_NDK_HOME=/android/ndk
%env ANDROID_NDK_URL=https://dl.google.com/android/repository/android-ndk-r19c-linux-x86_64.zip
%env ANDROID_SDK_FILENAME=tools_r25.2.5-linux.zip
%env ANDROID_SDK_HOME=/android/sdk
#%env ANDROID_HOME=/android/sdk
%env ANDROID_SDK_URL=https://dl.google.com/android/repository/tools_r25.2.5-linux.zip

#!echo $PATH
!export -p

In [ ]:
# Install specific versions of sdk, tools etc.
!android update sdk --no-ui -a \
  --filter tools,platform-tools,android-29,build-tools-29.0.2

## Install BAZEL with Baselisk

In [ ]:
# Download Latest version of Bazelisk
!wget https://github.com/bazelbuild/bazelisk/releases/latest/download/bazelisk-linux-amd64

# Make script executable
!chmod +x bazelisk-linux-amd64

# Adding to the path
!sudo mv bazelisk-linux-amd64 /usr/local/bin/bazel

# Extract bazel info
!bazel

In [ ]:
# Clone TensorFlow Lite Support repository OR upload your custom folder to build
!git clone https://github.com/tensorflow/tflite-support.git

In [ ]:
# Move into tflite-support folder
%cd /content/tflite-support/

!ls

## Build .aar files

In [ ]:
#@title Select library. { display-mode: "form" }

library = 'Support library' #@param ["Support library", "Task Vision library", "Task Text library", "Task Audio library","Metadata library","C++ image_classifier","C++ image_objector","C++ image_segmenter","C++ image_embedder","C++ nl_classifier","C++ bert_nl_classifier", "C++ bert_question_answerer", "C++ metadata_extractor"]

print('You selected:', library)

if library == 'Support library':
    library = '//tensorflow_lite_support/java:tensorflowlite_support.aar'
elif library == 'Task Vision library':
    library = '//tensorflow_lite_support/java/src/java/org/tensorflow/lite/task/vision:task-library-vision'
elif library == 'Task Text library':
    library = '//tensorflow_lite_support/java/src/java/org/tensorflow/lite/task/text:task-library-text'
elif library == 'Task Audio library':
    library = '//tensorflow_lite_support/java/src/java/org/tensorflow/lite/task/audio:task-library-audio'
elif library == 'Metadata library':
    library = '//tensorflow_lite_support/metadata/java:tensorflow-lite-support-metadata-lib'
elif library == 'C++ image_classifier':
    library = '//tensorflow_lite_support/cc/task/vision:image_classifier'
elif library == 'C++ image_objector':
    library = '//tensorflow_lite_support/cc/task/vision:image_objector'
elif library == 'C++ image_segmenter':
    library = '//tensorflow_lite_support/cc/task/vision:image_segmenter'
elif library == 'C++ image_embedder':
    library = '//tensorflow_lite_support/cc/task/vision:image_embedder'
elif library == 'C++ nl_classifier':
    library = '//tensorflow_lite_support/cc/task/text/nlclassifier:nl_classifier'
elif library == 'C++ bert_nl_classifier':
    library = '//tensorflow_lite_support/cc/task/text/nlclassifier:bert_nl_classifier'
elif library == 'C++ bert_question_answerer':
    library = '//tensorflow_lite_support/cc/task/text/qa:bert_question_answerer'
elif library == 'C++ metadata_extractor':
    library = '//tensorflow_lite_support/metadata/cc:metadata_extractor'



In [ ]:
#@title Select platform(s). { display-mode: "form" }

platforms = 'arm64-v8a,armeabi-v7a' #@param ["arm64-v8a,armeabi-v7a","x86", "x86_64", "arm64-v8a", "armeabi-v7a","x86,x86_64,arm64-v8a,armeabi-v7a"]
print('You selected:', platforms)


In [ ]:
# Build library
!bazel build \
    --fat_apk_cpu='{platforms}' \
    '{library}'